In [1]:
import pandas as pd
import statistics
MRR1 = []
MRR2 = []
MRR3 = []
pd.options.mode.chained_assignment = None
df = pd.read_csv('C:/Users/lucan/Documents/Uni/Courses/Bachelor Semester Project/BSP1/code/yagoFactsCleaned.csv')
df.columns = ['Subject', 'Predicate', 'Object']

The following function is similar to the code in "yagoFactsSearchEngine.py":

In [2]:
def search(queryList, goldenTriple):
    rpr = 0
    for l in queryList:
        mainIndexList = []
        importance = []
        predicateListFrequencies = []
        for i in range(0, len(queryList)):
            mainIndexList.extend(df.index[df.loc[:, "Subject"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Predicate"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Object"] == queryList[i]].tolist())
        mainIndexListUnique = list(dict.fromkeys(mainIndexList))
        for m in mainIndexListUnique:
            importance.append(100**mainIndexList.count(m))
        df2 = df.iloc[mainIndexListUnique]
        df2.loc[:, "Relevance"] = importance
        predicateList = df2.loc[:, 'Predicate'].tolist()
        predicateListUnique = list(dict.fromkeys(predicateList))
        for k in predicateList:
            predicateListFrequencies.append(predicateList.count(k))
        lowestUniquePredicateFrequency = min(predicateListFrequencies)
        for n in predicateListUnique:
            df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance'] = (df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance']/predicateList.count(n)) * lowestUniquePredicateFrequency
    df2 = df2.sort_values(by=['Relevance'], ascending=False)
    if len(df2) >= 10:
        for p in range(0, len(df2)):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    else:
        for p in range(0, 10):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    return rpr

The mean reciprocal rank (MRR) of this search function is:

In [3]:
%%time
MRR1.append(search(['Rocky_Johnson', 'Dwayne_Johnson'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR1.append(search(['Rocky_Johnson', 'hasChild'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR1.append(search(['Roman_Empire', 'hasCurrency'], ['Roman_Empire', 'hasCurrency', 'Sestertius']))
MRR1.append(search(['Rome', 'http://www.comune.roma.it/'], ['Rome', 'hasWebsite', 'http://www.comune.roma.it/']))
MRR1.append(search(['directed', 'San_Andreas_(film)'], ['Brad_Peyton', 'directed', 'San_Andreas_(film)']))
MRR1.append(search(['wroteMusicFor', 'Cosmopolitan_(film)'], ['Andrew_Lockington', 'wroteMusicFor', 'Cosmopolitan_(film)']))
MRR1.append(search(['Kiribati', 'hasCapital'], ['Kiribati', 'hasCapital', 'South_Tarawa']))
MRR1.append(search(['Charles_the_Fat', 'East_Francia'], ['Charles_the_Fat', 'wasBornIn', 'East_Francia']))
MRR1.append(search(['hasChild', 'Michelle_Obama'], ['Marian_Shields_Robinson', 'hasChild', 'Michelle_Obama']))
MRR1.append(search(['Greenland', 'hasCurrency'], ['Greenland', 'hasCurrency', 'Danish_krone']))
MRR1 = statistics.mean(MRR1)
print(MRR1)

0.35900429088611807
Wall time: 8min 59s


The MRR is **worse than expected** and the search function takes a lot of time. **Fixing the issues requires changing the ranking algorithm** to pay more attention to predicates in selected rows, instead of letting the function select all rows with a matching predicate and ranking those. The main changes in the following function are marked with comments.

In [4]:
def search(queryList, goldenTriple):
    rpr = 0
    for l in queryList:
        mainIndexList = []
        importance = []
        predicateListFrequencies = []
        for i in range(0, len(queryList)):
            mainIndexList.extend(df.index[df.loc[:, "Subject"] == queryList[i]].tolist()) # Omitting the search for
            mainIndexList.extend(df.index[df.loc[:, "Object"] == queryList[i]].tolist()) # rows with matching predicate
        mainIndexListUnique = list(dict.fromkeys(mainIndexList))
        for m in mainIndexListUnique:
            if df.iloc[m]["Predicate"] in queryList:
                importance.append(100**mainIndexList.count(m)*10) # Adding value to rows with matching predicates
            else:
                importance.append(100**mainIndexList.count(m))
        df2 = df.iloc[mainIndexListUnique]
        df2.loc[:, "Relevance"] = importance
        predicateList = df2.loc[:, 'Predicate'].tolist()
        predicateListUnique = list(dict.fromkeys(predicateList))
        for k in predicateList:
            predicateListFrequencies.append(predicateList.count(k))
        lowestUniquePredicateFrequency = min(predicateListFrequencies)
        for n in predicateListUnique:
            df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance'] = (df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance']/predicateList.count(n)) * lowestUniquePredicateFrequency
    df2 = df2.sort_values(by=['Relevance'], ascending=False)
    if len(df2) >= 10:
        for p in range(0, len(df2)):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    else:
        for p in range(0, 10):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    return rpr

The MRR of this function with the exact same benchmark queries is:

In [5]:
%%time
MRR2.append(search(['Rocky_Johnson', 'Dwayne_Johnson'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR2.append(search(['Rocky_Johnson', 'hasChild'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR2.append(search(['Roman_Empire', 'hasCurrency'], ['Roman_Empire', 'hasCurrency', 'Sestertius']))
MRR2.append(search(['Rome', 'http://www.comune.roma.it/'], ['Rome', 'hasWebsite', 'http://www.comune.roma.it/']))
MRR2.append(search(['directed', 'San_Andreas_(film)'], ['Brad_Peyton', 'directed', 'San_Andreas_(film)']))
MRR2.append(search(['wroteMusicFor', 'Cosmopolitan_(film)'], ['Andrew_Lockington', 'wroteMusicFor', 'Cosmopolitan_(film)']))
MRR2.append(search(['Kiribati', 'hasCapital'], ['Kiribati', 'hasCapital', 'South_Tarawa']))
MRR2.append(search(['Charles_the_Fat', 'East_Francia'], ['Charles_the_Fat', 'wasBornIn', 'East_Francia']))
MRR2.append(search(['hasChild', 'Michelle_Obama'], ['Marian_Shields_Robinson', 'hasChild', 'Michelle_Obama']))
MRR2.append(search(['Greenland', 'hasCurrency'], ['Greenland', 'hasCurrency', 'Danish_krone']))
MRR2 = statistics.mean(MRR2)
print(MRR2)

0.95
Wall time: 3min 42s


**The benchmark questions are carefully selected** to have exactly one correct answer, so the MRR still being below 1.0 is unexpected. **Due to a mistake**, the query below does in fact have two correct answers.

In [6]:
# Apart from the golden triple given as an argument in the function, there exists another correct answer:
# 'Chris_Rael', 'wroteMusicFor', 'Cosmopolitan_(film)'
print(search(['wroteMusicFor', 'Cosmopolitan_(film)'], ['Andrew_Lockington', 'wroteMusicFor', 'Cosmopolitan_(film)']))

0.5


This unintended result highlights a problem: **The search function returns one triple as the correct answer to the query, which is not ideal. For now, the search function will be modified to return single entities** (instead of triples) and compare them with a golden answer that corresponds to one entity. The idea is that the usual query contains two entities whose connection to each other is a third entity which is then returned as the answer.

In this case **the easiest way to proceed is to assume that the query won't contain predicates** - but only subjects and objects - so that the predicates are the 'edges' if the dataset is pictured as a knowledge graph. The following function looks for edges that are incident to the search terms and ranks them in importance to return the "answer" edge.

In [7]:
def search(queryList, goldenAnswer): # Change goldenTriple to goldenAnswer
    rpr = 0
    for l in queryList:
        mainIndexList = []
        importance = []
        predicateListFrequencies = []
        for i in range(0, len(queryList)):
            mainIndexList.extend(df.index[df.loc[:, "Subject"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Object"] == queryList[i]].tolist())
        mainIndexListUnique = list(dict.fromkeys(mainIndexList))
        for m in mainIndexListUnique:
            importance.append(100**mainIndexList.count(m)) # Change this part to ignore predicates in the query
        df2 = df.iloc[mainIndexListUnique]
        df2.loc[:, "Relevance"] = importance
        predicateList = df2.loc[:, 'Predicate'].tolist()
        predicateListUnique = list(dict.fromkeys(predicateList))
        for k in predicateList:
            predicateListFrequencies.append(predicateList.count(k))
        lowestUniquePredicateFrequency = min(predicateListFrequencies)
        for n in predicateListUnique:
            df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance'] = (df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance']/predicateList.count(n)) * lowestUniquePredicateFrequency
    df2 = df2.sort_values(by=['Relevance'], ascending=False)
    df2 = df2.loc[:, "Predicate"] # Make result table consist of predicates only
    if len(df2) >= 10:
        for p in range(0, len(df2)):
            if df2.iloc[p] == goldenAnswer[0]: # Compare with golden answer predicate
                rpr = 1/(p+1)
                break
    else:
        for p in range(0, 10):
            if df2.iloc[p] == goldenAnswer[0]: # "
                rpr = 1/(p+1)
                break
    return rpr

**The code below contains new benchmark questions** which don't take into account the possibility of multiple correct answers for one question worsening the RPR, **to give a less biased picture** (benchmark questions with predicates as search terms were removed from the sample queries to fulfil the assumption the new function makes about queries). 

Questions that contain two _not adjacent but connected_ nodes (subjects/objects) are also included to better compare this search function with later versions. The plan is that, at some point, the function will return several nodes and edges that represent the relationship between the search terms.

In [8]:
%%time
MRR3.append(search(['Barack_Obama', 'Marian_Shields_Robinson'], ['Michelle_Obama', 'hasChild', 'isMarriedTo']))
MRR3.append(search(['Sigmund_Freud', 'Kesswil'], ['Carl_Jung', 'hasAcademicAdvisor', 'wasBornIn']))
MRR3.append(search(['Battle_of_Talas', 'Tajikistan'], ['Kyrgyzstan', 'hasNeighbor', 'happenedIn']))
MRR3.append(search(['Ricochet_(TV_production_company)', 'Sahara'], ['Unbreakable_(TV_series)', 'created', 'isLocatedIn']))
MRR3.append(search(['Rocky_Johnson', 'Dwayne_Johnson'], ['hasChild']))
MRR3.append(search(['Rome', 'http://www.comune.roma.it/'], ['hasWebsite']))
MRR3.append(search(['Charles_the_Fat', 'East_Francia'], ['wasBornIn']))
MRR3.append(search(['Luxembourg', 'Luxembourg_City'], ['hasCapital']))
MRR3.append(search(['Toby_Barrett', 'Long_Point,_Ontario'], ['isLeaderOf']))
MRR3.append(search(['Metra', 'North_Central_Service'], ['owns']))
MRR3.append(search(['Gordon_Ramsay', 'Culinary_Genius_(TV_series)'], ['created']))
MRR3.append(search(['Kugelmugel', 'German_language'], ['hasOfficialLanguage']))
MRR3.append(search(['Yoshitami_Kuroiwa', 'Godzilla_1985'], ['edited']))
MRR3.append(search(['Gisborne_Airport', 'Auckland_Airport'], ['isConnectedTo']))
MRR3.append(search(['Macedonia_(ancient_kingdom)', 'Siege_of_Cyropolis'], ['participatedIn']))
MRR3.append(search(['Aristotle', 'Euboea'], ['diedIn']))
MRR3.append(search(['Latvia', 'Belarus'], ['hasNeighbor']))
MRR3.append(search(['Luigi_Ambrosio', 'Ennio_de_Giorgi'], ['hasAcademicAdvisor']))
MRR3.append(search(['Jeff_Bezos', 'Amazon.com'], ['created']))
MRR3.append(search(['Tatsuro_Yamashita', 'Ride_On_Time_(album)'], ['created']))
MRR3 = statistics.mean(MRR3)
print(MRR3)

0.75
Wall time: 7min 12s


A score of 0.75 for **the MRR is very good** considering that out of 20 benchmark queries, the first 4 cannot have anything but 0 as their RPR.

The good score is probably owning to the fact that the 'correct' answer doesn't have to be an entire triple anymore, it only needs to be the correct predicate. **However, the problem mentioned earlier isn't fixed yet.** The following example demonstrates this.

In [9]:
print(search(['Jeff_Bezos', 'Amazon.com'], ['created']))
print(search(['Jeff_Bezos', 'Amazon.com'], ['owns']))

1.0
0.5


Putting this aside and assuming that the MRR would go up to 0.8 with minor tweaking of the ranking algorithm, it is clear that the main weakness of the new search function is finding the relationship between search terms that don't appear together in any triple in the knowledge graph of the Yago dataset.